# Data Preparation, Pipelines & Model 

In [1]:
# Modules importeren
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# Dataset importeren 
df = pd.read_csv("/Users/odessa/Desktop/Applied Data Science & AI/Data Science/Code Inleiding data science/song_data.csv")

# Target variabele maken 
target = 'song_popularity'
#df.drop(columns=["song_name"], inplace=True) # inplace=True veranderd de originele dataframe zonder nieuwe dataframe te maken 

### Phase 3: Data Preparation

In [2]:
# 2 nummers droppen
df = df.drop([7119, 11171]).reset_index(drop=True)

In [3]:
print(f"Totaal aantal waardes in de dataframe vóór het verwijderen van dubbele waardes uit song_name en song_duration_ms: {len(df)}")

# Dubbele waardes droppen van song_name en song_duration 
# Als ik alleen song_name duplicates zou verwijderen, zou ik misschien covers van nummers verwijderen, dus daarom check ik ook de song_duration 
df.drop_duplicates(subset=['song_name', 'song_duration_ms'], inplace = True)
print(f"Totaal aantal waardes in de dataframe na verwijderen van dubbele waardes uit song_name en song_duration_ms: {len(df)}")

Totaal aantal waardes in de dataframe vóór het verwijderen van dubbele waardes uit song_name en song_duration_ms: 18833
Totaal aantal waardes in de dataframe na verwijderen van dubbele waardes uit song_name en song_duration_ms: 14466


In [4]:
df.drop(columns=["song_name"], inplace=True) # inplace=True veranderd de originele dataframe zonder nieuwe dataframe te maken 

In [5]:
X = df.drop(columns=[target], axis=1)
y = df[target]

### Phase 4: Modeling 

Supervised learning, omdat je de uitkomst al hebt 
<br>
Supervised learning heeft 2 hoofdtakken: regressie en classificatie 
<br>
RMSE 
<br>
Meervoudige lineare regressie 
<br>
Logistieke lineare regressie is classification 
<br>
Random forests is het begin van dat machine learning slim werd 

In [6]:
# BaseEstimator zorgt dat sklearn mijn class kan herkennen als model/stap in pipeline.
# TransformerMixin geeft .fit_transform().
class Winsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, kolommen): 
        self.kolommen = kolommen 
        self.grenzen_ = None # '_' betekent dat het attribuut pas beschikbaar wordt, nadat fit() is uitgevoerd. 
                             # None, omdat de grenzen nog niet bestaan -- worden berekend bij fit().

    def fit(self, X, y=None):
        """Bereken de onder- en bovengrenzen per kolom met interkwartielafstand-regel."""
        self.grenzen_ = {}
        for kolom in self.kolommen:
            Q1 = X[kolom].quantile(0.25)
            Q3 = X[kolom].quantile(0.75)
            IKR = Q3 - Q1 
            ondergrens = Q1 - 1.5 * IKR
            bovengrens = Q3 + 1.5 * IKR
            self.grenzen_[kolom] = (ondergrens, bovengrens)
        return self 
    
    def transform(self, X):
        """Winsoriseer uitschieters: vervang alle waardes buiten de grenzen met de dichtstbijzijnde grenswaarde."""
        X = X.copy() # Kopie maken van data 
        for kolom, (ondergrens, bovengrens) in self.grenzen_.items():
            X.loc[X[kolom] < ondergrens, kolom] = ondergrens 
            X.loc[X[kolom] > bovengrens, kolom] = bovengrens 
        return X

In [7]:
def nieuwe_features(X):
    X = X.copy()
    df['energy_dance'] = df['energy'] * df['danceability']
    df['tempo_loudness'] = df['tempo'] * df['loudness']
    df['valence_energy'] = df['audio_valence'] * df['energy']
    df['acoustic_energy_ratio'] = df['acousticness'] / (df['energy'] + 0.001)
    df['duration_min'] = df['song_duration_ms'] / 60000
    return X

feature_engineering = FunctionTransformer(nieuwe_features, validate=False)

In [8]:
# Kolommen indelen
kolommen_winsoriseren = ['song_duration_ms', 'loudness', 'tempo', 'time_signature']
categorische_kolommen = ['key', 'audio_mode']

#Alle nummerieke kolommen behalve target
numerieke_kolommen = X.select_dtypes(include=['int64', 'float64'])
overige_kolommen = [
    c for c in numerieke_kolommen 
    if c not in kolommen_winsoriseren + categorische_kolommen
]

In [9]:
# Preprocessing

preprocessor = ColumnTransformer([
    ('winsor_scale', Pipeline ([
        ('winsor', Winsorizer(kolommen=kolommen_winsoriseren)),
        ('scaler', StandardScaler())
    ]), kolommen_winsoriseren),

    ('scale_rest', StandardScaler(), overige_kolommen),

    ('onehot', OneHotEncoder(drop='first', sparse_output=False), categorische_kolommen) # sparse_output = False zorgt ervoor dat de output een gewone DataFrame-array is, nodig voor lineaire regressie.
])

In [10]:
pipeline = Pipeline([
    ('feature_creation', feature_engineering),
    ('preprocess', preprocessor),
    ('model', LinearRegression())
])

In [11]:
# Train en test set maken 
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state = 42
)

In [12]:
# Trainen en evalueren 
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(f'R²: {r2_score(y_test, y_pred):.3f}')
print(f'MSE: {mean_squared_error(y_test, y_pred):.3f}')

R²: 0.026
MSE: 414.369
